In [ ]:
!pip install ipywidgets

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

In [ ]:
!unzip annotations_trainval2017.zip

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
import requests
from io import BytesIO
from tqdm import tqdm

In [ ]:
with open("/content/annotations/captions_train2017.json", "r") as f:
  data = json.load(f)
data.keys()

In [ ]:
keywords = ["road", "cars"]
result_list = []
for item in data['annotations']:
    for keyword in keywords:

        if keyword in item['caption'].lower():
            result_list.append(item)
            continue

print(len(result_list))

In [ ]:
dict_id_captions = {}
for _data in result_list:
    if(dict_id_captions.get(_data['image_id']) is None):
        dict_id_captions[ _data['image_id'] ] = []
    dict_id_captions[ _data['image_id'] ].append(_data['caption'])

In [ ]:
image_with_caption = {}
for image in data['images']:
    captions = dict_id_captions.get(image['id'])
    if(captions is not None):
        if( len(captions) > 0):
            image_with_caption[image['id']] = {"url": image['coco_url'], "captions": captions}

In [ ]:
class DataLooker():
    def __init__(self, data):
        self.data = data
        self.key_list = list( self.data.keys())
        self.tem = []

    def showNImageAndCaption(self, from_index, to_index):
        for _key in self.key_list[from_index:to_index]:
            image = self.data.get(_key)
            image['id'] = _key
            self.tem.append( image )
        return self

    def drawIteratively(self):
        # 이미지와 캡션을 담을 위젯 리스트 생성
        items = []

        for image in tqdm(self.tem):
            response = requests.get(image['url'])
            img_data = BytesIO(response.content).getvalue()

            # 이미지 위젯 생성
            img_widget = widgets.Image(value=img_data, format='jpeg', width=300, height=300)

            # 캡션이 있을 경우 처리
            if image.get('captions'):
                if len(image['captions']) > 0:
                    # HTML 태그를 사용하여 줄바꿈을 삽입
                    captions_text = "<br>".join(image['captions'])
            else:
                captions_text = "<<No captions available.>>"

            # 캡션 HTML 위젯 생성
            caption_widget = widgets.HTML(value=f"<p>{image['id']}: {captions_text}</p>")

            # 이미지와 캡션을 수직으로 배치
            vbox = widgets.VBox([img_widget, caption_widget])

            # GridBox에 넣을 항목 추가
            items.append(vbox)

        # 2열 그리드 박스 생성
        grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 1fr)"))

        # 출력
        display(grid)

In [ ]:
datalookwer = DataLooker(image_with_caption)
datalookwer.showNImageAndCaption(0, 106).drawIteratively()